In [1]:
from vismanip import TransitVis
import struct
import datetime
import pytz
import numpy as np

In [1]:
fvis = open('/project/druhe/SB281-202011112023-lba_outer.vis', 'rb')

In [16]:
NUM_ANT = 576

In [3]:
LEN_HDR = 512
nant = 576
nbline  = nant*(nant+1)/2
pol2rec = 2
# 8 due to float complex.
nbline = int(nant * (nant + 1) / 2)
nchan = 3
payload = nbline * nchan * pol2rec * 8
recsize = int(payload + LEN_HDR)
nweight =(12*13)/2
sub = 281
freq = sub * 195312.5
npol = 2
norigchan = nchan
ateamflux       = np.zeros ( (npol, 5) ) 


In [4]:
def parse_vis_hdr(hdr):
    magic, t0, t1 = struct.unpack("<Qdd", hdr[0:24])
    print(magic, t0, t1)
    weights = struct.unpack (("<%di" % nweight), 
                                header[148:int(148+nweight*4)])
    return (magic, datetime.datetime.utcfromtimestamp(t0).replace(tzinfo=\
        pytz.utc), datetime.datetime.utcfromtimestamp(t1).replace(tzinfo=\
        pytz.utc), sub, nant, npol, norigchan, 
        ateamflux, 0, 0, weights, freq)

In [5]:
recbuf = fvis.read(recsize)

In [6]:
header = recbuf[0:512]
data = recbuf[512:]

In [12]:
m, trec, t1, subband,_,_,_,_,_,_,_,_ = parse_vis_hdr(header)


1081992285376802818 1605126360.0 1605126361.006633


In [7]:
from scipy.constants import c

def parse_lba_antennafield(f, subband):

    with open(f) as f:
        lines = [line.strip() for line in f.readlines()]

    positions = []
    lba_start = lines.index('LBA')
    data_start = lba_start + 3
    data_end = lines.index(']', lba_start)
    num_antennas = data_end - data_start
    offset = [float(x) for x in lines[lba_start+1].split()[2:5]]
    for line in lines[data_start: data_end]:
        x, y, z = [float(x) for x in line.split()[0:3]]
        positions.append(
            [offset[0] + x, offset[1] + y, offset[2] + z]
        )
    
    rot_matrix = []
    lba_rot_start = lines.index('ROTATION_MATRIX LBA')
    data_start = lba_rot_start + 2
    data_end = lba_rot_start + 5
    for line in lines[data_start: data_end]:
        x, y, z = [float(x) for x in line.split()[0:3]]
        rot_matrix.append(
            [x, y, z]
        )

    A = np.array(positions)
    
    np.savetxt('lba_outer_a12.dat', A, fmt='%f')
    
    raise
    R = np.array(rot_matrix)
    L = A.dot(R)
    print(len(L))

    u, v = [], []

    for a1 in range(0, num_antennas):
        for a2 in range(0, a1+1):
            u.append(L[a1, 0] - L[a2, 0])
            v.append(L[a1, 1] - L[a2, 1])

    # Don't know what these constants are. Probably converting to wavelengths. 
    return [np.ravel([(np.array(u)/(c/(s*(2e8/1024))/2.0)) for s in subbands]),
            np.ravel([(np.array(v)/(c/(s*(2e8/1024))/2.0)) for s in subbands])]


In [8]:
import matplotlib.pyplot as plt


In [9]:
antennafield = parse_lba_antennafield('a12-AntennaField.conf', 288)
plt.scatter(*antennafield, s=1)
plt.xlabel('U')
plt.ylabel('V')
plt.title('AARTFAAC12 UV Coverage')
plt.savefig('a12-coverage.pdf')

RuntimeError: No active exception to reraise

In [4]:
import matplotlib.pyplot as plt
antennafield = parse_lba_antennafield('a6-AntennaField.conf', 288)
plt.scatter(*antennafield, s=1)
plt.xlabel('U')
plt.ylabel('V')
plt.title('AARTFAAC6 UV Coverage')
plt.savefig('a6-coverage.pdf')

TypeError: _savetxt_dispatcher() missing 1 required positional argument: 'X'

In [103]:
m, t0, t1, s, d, p, c, fl, *_ = parse_vis_hdr(header)

1081992285376802818 1605126360.0 1605126361.006633


In [104]:
fl

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [114]:
def parse_header(hdr):
    """
    Parse aartfaac header for calibrated data
    struct output_header_t
    {
      uint64_t magic;                   ///< magic to determine header                (  8 B)
      double start_time;                ///< start time (unix)                        (  8 B)
      double end_time;                  ///< end time (unix)                          (  8 B)
      int32_t subband;                  ///< lofar subband                            (  4 B)
      int32_t num_dipoles;              ///< number of dipoles (288 or 576)           (  4 B)
      int32_t polarization;             ///< XX=0, YY=1                               (  4 B)
      int32_t num_channels;             ///< number of channels (<= 64)               (  4 B)
      float ateam_flux[5];              ///< Ateam fluxes (CasA, CygA, Tau, Vir, Sun) ( 24 B)
      std::bitset<5> ateam;             ///< Ateam active                             (  8 B)
      std::bitset<64> flagged_channels; ///< bitset of flagged channels               (  8 B)
      std::bitset<576> flagged_dipoles; ///< bitset of flagged dipoles                ( 72 B)
      uint32_t weights[78];             ///< stationweights n*(n+1)/2, n in {6, 12}   (312 B)
      uint8_t pad[48];                  ///< 512 byte block                           ( 48 B)
    };
    """
    m, t0, t1, s, d, p, c = struct.unpack("<Qddiiii", hdr[0:40])
    # Flagged dipoles.
    f = np.frombuffer(hdr[80:152], dtype=np.uint64)
    return (m, t0, t1, s, d, p, c, f)


In [115]:
m, t0, t1, s, d, p, c, fl = parse_header(header)


In [116]:
flagged = []
for j,v in enumerate(fl):
    for k in range(64):
        if np.bitwise_and(v, np.uint64(1<<k)):
            flagged.append(j*64+k)


In [118]:
len(flagged)

108

In [120]:
size = os.path.getsize(fvis.name)


In [131]:
NUM_ANT = 576
NUM_BSL = (NUM_ANT ** 2 + NUM_ANT ) // 2
LEN_HDR = 512
LEN_BDY = NUM_BSL * 8

N = size / (LEN_BDY + LEN_HDR)

In [135]:
size

5711320064

In [136]:
LEN_BDY

1329408

In [133]:
for i in range(N):
    pass

TypeError: 'float' object cannot be interpreted as an integer

In [134]:
NUM_BSL

166176

In [ ]:
(n-1) * (LEN_BDY + LEN_HDR)

In [181]:
def align_to_magic(fp):
    magic = 0 
    stream = [b'\x00'] * 8
    while magic != HDR_MAGIC:
        stream.append(fp.read(1))
        stream.pop(0)
        magic = struct.unpack('Q', b''.join(stream))[0]
        print(magic)

        break

    return fp.tell() - 8


In [182]:
align_to_magic(fvis)

0


7976972

In [139]:
HDR_MAGIC = 0x4141525446414143

In [140]:
HDR_MAGIC

4702130007860920643